# Projeto 8 - Detecção de Pragas com Imagens Agrícolas usando Inteligência Artificial

A  ferrugem  da  folha  do  trigo  é  a  doença  mais  comum  desta cultura.  As  perdas  em rendimento de grãos podem chegar a 50%.Essa doença  manifesta-se desde o surgimento das  primeiras folhas até a maturação da planta.  

Inicialmente,  surgem  pequenos  nódulos  arredondados,  amarelo-alaranjados,  dispostos sem  ordenação  (parecendo  ferrugem  mesmo),  normalmente  localizados  na  face  superior  das folhas, estendendo-se ao caule. Estas frutificações ficam sempre recobertas pela epiderme até o final do ciclo da planta.

O patógeno sobrevive no verão-outono parasitando plantas de trigo que se constituem na principal fonte de problemas em plantações de trigo em países mais quentes, como o Brasil. As condições ambientais para o desenvolvimento da doença são temperatura média de 20ºC e mais de 6 horas de molhamento foliar contínuo.

O  objetivo  deste  Projeto  é  construir  um  modelo  de  Deep  Learning  para  classificar corretamente se uma planta (trigo) é saudável, possui ferrugem no caule ou ferrugem na folha.

Fonte dos dados: https://zindi.africa/competitions/iclr-workshop-challenge-1-cgiar-computer-vision-for-crop-disease/data

## 1. Instalando e carregando os pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Pacote de utilitários
!pip install -q imutils

In [3]:
# Comando para silenciar o Keras
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [4]:
# Imports

# Pacotes para manipulação e visualização de dados
import os
import cv2
import pickle
import imutils
import random
import sklearn
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from imutils import paths
from random import randint
from collections import defaultdict

# Pacotes para Deep Learning
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import load_model, save_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow.keras.optimizers import Adam 

# Pacotes para processamento de dados e avaliação do modelo
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# Define a área de plotagem das imagens
from IPython.display import clear_output
plt.rcParams['figure.figsize'] = (15, 9)
%matplotlib inline

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Projeto 08 - Detecção de Pragas com Imagens Agrícolas" --iversions

Author: Projeto 08 - Detecção de Pragas com Imagens Agrícolas

matplotlib: 3.5.2
keras     : 2.11.0
cv2       : 4.7.0
imutils   : 0.5.4
sklearn   : 1.0.2
numpy     : 1.21.5
tensorflow: 2.11.0



## 2. Carga das imagens de treino

In [6]:
# Imagens de treino
imagens_treino = "dados/train"

# Imagens de teste
imagens_teste = "dados/test"

In [7]:
# Caminho das imagens
caminho_imagens_treino = sorted(list(paths.list_images(imagens_treino)))

In [8]:
# Suffle das imagens
random.shuffle(caminho_imagens_treino)

In [9]:
# Lista para armazenar as imagens
dados = []

# Lista para armazenar os labels
labels = []

# Dimensões das imagens
image_dims = (224, 224, 3)

In [10]:
# Loop pelos caminhos das imagens de treino

print("\nIniciando o processamento das imagens de treino. Aguarde.")

count = 0

for caminho in caminho_imagens_treino:
    
    # Leitura da imagem
    image = cv2.imread(caminho)
    
    # Redimensionamento
    # Argumento cv2.INTER_AREA é para realizar a interpolação das imagens
    # Isso é necessário porque nem todas as imagens podem ficar com 224 x 224
    image = cv2.resize(image, (image_dims[1], image_dims[0]), cv2.INTER_AREA)
    
    # Converte a imagem para array numpy
    image = img_to_array(image)
    
    # Adiciona o array da imagem à lista de arrays
    dados.append(image)

    # Extrai o label das classes
    label = caminho.split(os.path.sep)[-2]

    # Adiciona o label à lista de labels
    labels.append(label)
    
    # Atualiza o contador
    count += 1

print("\nProcessamento Concluído. Total de imagens processadas:", count)


Iniciando o processamento das imagens de treino. Aguarde.

Processamento Concluído. Total de imagens processadas: 562


In [11]:
# Padroniza a intensidade dos pixels para o range [0,1]
dados = np.array(dados, dtype = 'float') / 255.0

In [12]:
# Convertemos os labels para o formato de array numpy
labels = np.array(labels)

In [13]:
# Conversão dos labels em binário
# Amostra dos labels
labels[1:10]

array(['stem_rust', 'stem_rust', 'leaf_rust', 'leaf_rust', 'stem_rust',
       'leaf_rust', 'stem_rust', 'leaf_rust', 'leaf_rust'], dtype='<U13')

In [14]:
# Cria o binarizador
binarizador = LabelBinarizer()

In [15]:
# Aplica o binarizador aos labels
labels = binarizador.fit_transform(labels)

In [16]:
# Visualiza os labels
labels[1:10]

array([[0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0]])

In [17]:
# Salva o binarizador, para usar nos dados de teste
obj_binarizador = open("modelos/binarizador.pickle", "wb")
obj_binarizador.write(pickle.dumps(binarizador))
obj_binarizador.close()

In [18]:
# Divisão dos dados em treino e teste
(X_treino, X_teste, y_treino, y_teste) = train_test_split(dados, labels, test_size = 0.2)

In [19]:
# Shape dos dados de treino e teste
print("Shape de X_treino:", X_treino.shape)
print("Shape de y_treino:", y_treino.shape)
print("Shape de X_teste:", X_teste.shape)
print("Shape de y_teste:", y_teste.shape)

Shape de X_treino: (449, 224, 224, 3)
Shape de y_treino: (449, 3)
Shape de X_teste: (113, 224, 224, 3)
Shape de y_teste: (113, 3)


## 3. Construção do modelo

Será utilizado um modelo com Deep Convolutional Network, usando Transferência de Aprendizado.

O modelo base será o InceptionV3 pré-treinado e fornecido pelo Keras e sobre ele será adicionado as camadas.

Referências:

https://keras.io/api/applications/inceptionv3/

http://www.deeplearningbook.com.br/

In [20]:
# Função para criar o modelo
def cria_modelo(use_imagenet = True):
    
    # Carrega o modelo pré-treinado, mas sem as camadas finais, por isso include_top = False
    # As camadas finais adicionaremos ao modelo mais abaixo
    modelo_base = (keras.applications.InceptionV3(include_top = False,
                                                  input_shape = image_dims,
                                                  weights = 'imagenet' if use_imagenet else None))
    
    # Adicionamos a camada de Global Pooling
    novo_modelo = keras.layers.GlobalAveragePooling2D()(modelo_base.output)
    
    # Adicionamos a camada Densa de saída para as classes
    novo_modelo = keras.layers.Dense(len(binarizador.classes_), activation = 'softmax')(novo_modelo)
    
    # Concatena tudo para gerar o modelo que será usado
    modelo = keras.engine.training.Model(modelo_base.inputs, novo_modelo)
    
    return modelo

In [21]:
# Inicializamos o modelo
modelo_final = cria_modelo()

87910968/87910968 [==============================] - 5s 0us/step


In [22]:
# Sumário do modelo
modelo_final.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 batch_normalization_5 (BatchNo  (None, 25, 25, 64)  192         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 25, 25, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_10 (BatchN  (None, 25, 25, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 25, 25, 32)  96          ['conv2d_11[0][0]']              
 ormalizat

                                                                                                  
 batch_normalization_22 (BatchN  (None, 25, 25, 64)  192         ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_22 (Activation)     (None, 25, 25, 64)   0           ['batch_normalization_22[0][0]'] 
                                                                                                  
 conv2d_20 (Conv2D)             (None, 25, 25, 48)   13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_23 (Conv2D)             (None, 25, 25, 96)   55296       ['activation_22[0][0]']          
                                                                                                  
 batch_nor

                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 12, 12, 288)  0          ['mixed2[0][0]']                 
                                                                                                  
 mixed3 (Concatenate)           (None, 12, 12, 768)  0           ['activation_26[0][0]',          
                                                                  'activation_29[0][0]',          
                                                                  'max_pooling2d_2[0][0]']        
                                                                                                  
 conv2d_34 (Conv2D)             (None, 12, 12, 128)  98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_34 (BatchN  (None, 12, 12, 128)  384        ['conv2d_34[0][0]']              
 ormalizat

                                                                  'activation_39[0][0]']          
                                                                                                  
 conv2d_44 (Conv2D)             (None, 12, 12, 160)  122880      ['mixed4[0][0]']                 
                                                                                                  
 batch_normalization_44 (BatchN  (None, 12, 12, 160)  480        ['conv2d_44[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_44 (Activation)     (None, 12, 12, 160)  0           ['batch_normalization_44[0][0]'] 
                                                                                                  
 conv2d_45 (Conv2D)             (None, 12, 12, 160)  179200      ['activation_44[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 activation_54 (Activation)     (None, 12, 12, 160)  0           ['batch_normalization_54[0][0]'] 
                                                                                                  
 conv2d_55 (Conv2D)             (None, 12, 12, 160)  179200      ['activation_54[0][0]']          
                                                                                                  
 batch_normalization_55 (BatchN  (None, 12, 12, 160)  480        ['conv2d_55[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_55 (Activation)     (None, 12, 12, 160)  0           ['batch_normalization_55[0][0]'] 
          

                                                                                                  
 batch_normalization_65 (BatchN  (None, 12, 12, 192)  576        ['conv2d_65[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_65 (Activation)     (None, 12, 12, 192)  0           ['batch_normalization_65[0][0]'] 
                                                                                                  
 conv2d_61 (Conv2D)             (None, 12, 12, 192)  147456      ['mixed6[0][0]']                 
                                                                                                  
 conv2d_66 (Conv2D)             (None, 12, 12, 192)  258048      ['activation_65[0][0]']          
                                                                                                  
 batch_nor

                                                                                                  
 conv2d_70 (Conv2D)             (None, 12, 12, 192)  147456      ['mixed7[0][0]']                 
                                                                                                  
 conv2d_74 (Conv2D)             (None, 12, 12, 192)  258048      ['activation_73[0][0]']          
                                                                                                  
 batch_normalization_70 (BatchN  (None, 12, 12, 192)  576        ['conv2d_70[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_74 (BatchN  (None, 12, 12, 192)  576        ['conv2d_74[0][0]']              
 ormalization)                                                                                    
          

 batch_normalization_76 (BatchN  (None, 5, 5, 320)   960         ['conv2d_76[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_78 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_78[0][0]'] 
                                                                                                  
 activation_79 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_79[0][0]'] 
                                                                                                  
 activation_82 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_82[0][0]'] 
                                                                                                  
 activation_83 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_83[0][0]'] 
          

                                                                                                  
 activation_91 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_91[0][0]'] 
                                                                                                  
 activation_92 (Activation)     (None, 5, 5, 384)    0           ['batch_normalization_92[0][0]'] 
                                                                                                  
 batch_normalization_93 (BatchN  (None, 5, 5, 192)   576         ['conv2d_93[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_85 (Activation)     (None, 5, 5, 320)    0           ['batch_normalization_85[0][0]'] 
                                                                                                  
 mixed9_1 

#### BatchNormalization

Batch Normalization é uma técnica utilizada em Deep Learning para melhorar a eficiência e a estabilidade durante o treinamento de redes neurais. Foi introduzida por Sergey Ioffe e Christian Szegedy em 2015.

O principal objetivo da Batch Normalization é resolver o problema do "covariate shift", que se refere a mudanças nas distribuições de entrada para as camadas de uma rede neural durante o treinamento. Como o aprendizado de cada camada depende das camadas anteriores, pequenas mudanças nos pesos podem levar a grandes mudanças na distribuição dasentradaspara camadas posteriores, fazendo com que o modelo tenha que se adaptar constantemente a estas novas distribuições.

O modelo já está pré-treinado, mas podemos alterar alguns determinados parâmetros. As mudanças são do BatchNormalization, conforme o código abaixo

In [23]:
# Loop por todas as camadas do modelo
for layer in modelo_final.layers:
    
    # camada treinável
    layer.trainable = True
    
    # Fazemos suavização exponencial agressiva dos pesos nas camadas de Batch Normalization 
    # para treinar o modelo mais rapidamente.
    if isinstance(layer, keras.layers.BatchNormalization):
        layer.momentum = 0.7

In [24]:
# Para as camadas mais profundas não fazemos a suavização exponencial (somente as últimas 50)
for layer in modelo_final.layers[:-50]:
    
    # Se a camada não for de Batch Normalization não será treinável
    if not isinstance(layer, keras.layers.BatchNormalization):
        layer.trainable = False

In [27]:
# Caminho para salvar os checkpoints do modelo
caminho_modelo = "modelos/modelo_treinado.hdf5"

#### Early stopping

Early stopping é uma técnica de regularização usada durante o treinamento de redes neurais para prevenir o overfitting. Overfitting acontece quando um modelo aprende o conjunto de treinamento tão bem que tem um desempenho pobre quando exposto a dados novos, não vistos durante o treinamento.O early stopping funciona monitorando a performance do modelo em um conjunto de validação durante o treinamento. Se a performance parar de melhorar após umdeterminado número de épocas (ou até começar a piorar), o treinamento é interrompido, ou seja, fazemos um  "early  stopping"(parada  antecipada).  O  modelo  salvo  é  então  o  que  teve  a  melhor performance no conjunto de validação.

In [28]:
# Vamos monitorar o erro durante o treinamento e salvar sempre a melhor versão do modelo
checkpoint1 =  ModelCheckpoint(caminho_modelo, 
                               monitor = "val_loss", 
                               verbose = 1, 
                               save_best_only = True, 
                               mode = "min")

In [29]:
# Vamos monitorar o erro durante o treinamento e interromper o treinamento se depois de 8 épocas o erro não diminuir
checkpoint2 = EarlyStopping(monitor = 'val_loss', 
                            mode = 'min', 
                            verbose = 1, 
                            patience = 8)

In [30]:
# Lista de checkpoints
callbacks_list = [checkpoint1, checkpoint2]

## 4. Treino do modelo

In [31]:
# Hiperparâmetros

# Número de épocas de treinamento
epochs = 25 

# Taxa de aprendizado
taxa = 1e-3 

# Batch size
batch = 32 

In [32]:
# Compila o modelo
modelo_final.compile(optimizer = Adam(learning_rate = taxa), # Melhor otimizador em geral
                     loss = 'categorical_crossentropy', # função de erro para classificação binário
                     metrics = ['accuracy']) # Métrica de avaliação do modelo

#### Treino do modelo

In [34]:
%%time

print("\nIniciando o treinamento do modelo. Aguarde.\n")

history = modelo_final.fit(X_treino, y_treino, batch_size = batch,
                           validation_data = (X_teste, y_teste),
                           epochs = epochs, 
                           verbose = 1, 
                           callbacks = callbacks_list)

print("\nTreinamento Concluído.\n")


Iniciando o treinamento do modelo. Aguarde.

Epoch 1/25
15/15 [==============================] - ETA: 0s - loss: 0.9894 - accuracy: 0.6637
Epoch 1: val_loss improved from inf to 1.68647, saving model to modelos\modelo_treinado.hdf5
15/15 [==============================] - 53s 3s/step - loss: 0.9894 - accuracy: 0.6637 - val_loss: 1.6865 - val_accuracy: 0.7168
Epoch 2/25
15/15 [==============================] - ETA: 0s - loss: 0.2077 - accuracy: 0.9421
Epoch 2: val_loss improved from 1.68647 to 0.92215, saving model to modelos\modelo_treinado.hdf5
15/15 [==============================] - 49s 3s/step - loss: 0.2077 - accuracy: 0.9421 - val_loss: 0.9222 - val_accuracy: 0.7611
Epoch 3/25
15/15 [==============================] - ETA: 0s - loss: 0.1129 - accuracy: 0.9755
Epoch 3: val_loss did not improve from 0.92215
15/15 [==============================] - 49s 3s/step - loss: 0.1129 - accuracy: 0.9755 - val_loss: 1.0629 - val_accuracy: 0.7345
Epoch 4/25
15/15 [==============================

## 4. Avaliação do modelo

In [35]:
# Carrega o modelo
modelo = load_model("modelos/modelo_treinado.hdf5")

In [36]:
# Carrega o binarizador
obj_bin = pickle.loads(open("modelos/binarizador.pickle", "rb").read())

In [37]:
# Obtemos uma imagem de teste de forma randômica
imagem_teste = random.choice(os.listdir(imagens_teste))

In [38]:
# Imagem selecionada
imagem_teste

'TVPKJX.jpg'

In [39]:
# Fazemos a leitura da imagem com OpenCV
imagem = cv2.imread(os.path.join(imagens_teste, imagem_teste))

In [40]:
# Vamos criar uma cópia da imagem, pois usaremos a imagem original mais tarde
imagem_copia = imagem.copy()

In [41]:
# Aplicamos na imagem de teste o mesmo pré-processamento aplicado nas imagens de treino
imagem = cv2.resize(imagem, (224, 224))
imagem = imagem.astype("float") / 255.0
imagem = img_to_array(imagem)
imagem = np.expand_dims(imagem, axis = 0)

In [42]:
# E então fazemos a previsão com o modelo, extraindo a probabilidade de classe
proba = modelo.predict(imagem)[0]

1/1 [==============================] - 1s 774ms/step


In [43]:
# Para cada classe, temos a probabilidade e como temos 3 classes, são 3 probabilidades.
# A maior probabilidade indica a classe prevista pelo modelo
proba

array([6.0371129e-04, 2.4131963e-01, 7.5807667e-01], dtype=float32)

In [44]:
# Como queremos a classe prevista pelo modelo, obtemos o maior valor entre as 3 probabilidades
idx = np.argmax(proba)

In [45]:
# E aplicamos o binarizador ao label
label = obj_bin.classes_[idx]

In [46]:
# Vamos adicionar ao label a probabilidade
label = "{}: {:.2f}%".format(label, proba[idx] * 100) 

In [47]:
# Visualiza a previsão do modelo
label

'stem_rust: 75.81%'